# **Basics of Image Denoising using Autoencoders**

Hi :) 
This is my first notebook that I prepared to share and help others learn.

I have recently done some courses and I am putting my knowledge to practical use !!!

Open to suggestions !!!
Hope you like it.

Please note that I have focused on denoising and not really on analysis of results etc. using confusion matrix and other techniques

This notebook is not aimed at getting the best results, more fine tuning of the networks will definitely improve the results :)

This notebook has been coded using TensorFlow 2.0 and Python 3

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Importing Libraries**

In [ ]:
import seaborn as sns #data visualization

from sklearn.model_selection import train_test_split #data preprocessing to divide our data

import matplotlib.pyplot as plt #to plot the images of numbers
from tensorflow.keras.models import Sequential, Model #for building neural network
from tensorflow.keras.layers import Dense, Input #for layers of neural network
from tensorflow.keras.utils import to_categorical 
#plot in the notebook itself
%matplotlib inline 
sns.set(style = 'white', context = 'notebook', palette = 'deep')
np.random.seed(42)

# **Loading Data into Train and Test**

In [ ]:
#using pandas.read_csv we read the csv data into a pandas dataframe

train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")


# Some data analysis

In [ ]:
#train.head() shows us the first five columns of a dataframe
train.head()

1. From above we observe that data is stored such a way that a row represents one image and the pixels of the image are stored from pixel0 to pixel783. Which means that there are 784 pixels i.e image size is of 28x28 pixels. 
2. The label column represents the class of the image if it is a one, two, three ... or so on. We have a total of 10 classes (0-9)

**Understanding how to process data for our aim**
1. Our aim is to predict the class of an image, therefore we are going to store the labels column from the datafram in y_train.
2. To store x_train we will first drop the "label" column from the train dataframe and then proceed cast the dataframe to a new variable x_train


In [ ]:

y_train = train["label"]
x_train = train.drop(labels = ["label"], axis = 1)


#sns.countplot will count the number of unique values in y_train and give us the distribution in the form of a graph
graph = sns.countplot(y_train)

y_train.value_counts()

# Data Preprocessing

In [ ]:
print(type(x_train))
print(type(y_train))

We see that x_train is in the form of a pandas Dataframe and y_train in the form of pandas series, We are going to convert it into numpy to introduce noise in it later.

In [ ]:
x_train = x_train.to_numpy()
y_train = y_train.to_numpy()

print(type(x_train))
print(type(y_train))

In [ ]:
#Data Normalization 
x_train = x_train.astype('float')/255.

In [ ]:
# Splitting our data into training and testing
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, random_state = 42)

In [ ]:

#Number of the samples we have
x_train_size = len(x_train)
x_val_size = len(x_val)

print(x_train_size)
print(x_val_size)

# Adding Noise

* We are going to artifically add noise to our data.
* To do that we simpy add random data to our existing data.
* Recall that we had earlier normalid out data to be  between 0 and 1 by diving it by 255. Therefore the noise added should also be between 0 and 1. To do that we simply multiply the random noise generated by 0.9
* After adding the noise we clip the data to be between 0 and 1 again.


In [ ]:

x_train_noisy = x_train + np.random.rand(x_train_size, 784) * 0.9
x_val_noisy = x_val + np.random.rand(x_val_size, 784) * 0.9

#clipping the noise under 0 and 1 incase something went above
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_val_noisy = np.clip(x_val_noisy, 0., 1.)

**Defining a function to plot the images of numbers**


In [ ]:
def plot(x, p , labels = False):
    plt.figure(figsize = (20,2))
    for i in range(10):
        plt.subplot(1, 10, i+1)
        plt.imshow(x[i].reshape(28,28), cmap = 'binary') #reshaping the to conver 784 to 28x28 for an image plotting
        plt.xticks([])
        plt.yticks([])
        if labels:
            plt.xlabel(np.argmax(p[i]))
    plt.show()
    return
plot(x_train, None)

    

See the how the noise has been introduced to our dataset

In [ ]:
plot(x_train_noisy, None)

# Building and Training a Classifier

Constructing a very simple neural network to classify our images.

In [ ]:
classifier = Sequential([
    Dense(512, activation = 'relu', input_shape = (784, )),
    Dense(512, activation = 'relu'),
    Dense(256, activation = 'relu'),
    Dense(10, activation = 'softmax')
])

#using spare_categorical_crossentropy because labels have not been one-hot encoded
classifier.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = 'accuracy')

In [ ]:
classifier.summary()

Proceeding to train the classifier

In [ ]:
classifier.fit(x_train, y_train, epochs = 10, batch_size = 512)

In [ ]:
loss, acc = classifier.evaluate(x_val, y_val)

The classifier acheives good accuracy on images with no noise. But what about images with noise in them? The follwoing cell will tell

In [ ]:
loss, acc = classifier.evaluate(x_val_noisy, y_val)

* As you can see the accuracy of the model decreased manifold because of noise. The model is confused about making predictions because noise has rendered it unusable. Real world data is like that, there is lot of noise in images and if we want to make neural networks and deploy them in making predictions, then they need to be robust to noise too. 
* This is where autoencoders come into play.

# What are Autoencoders?

* Autoencoders are unsupervised neural netowrks which learn how to encode data and then reconstruct the data from reduced encodings to a representation as close to the original data.
* The consist of and **encoder**, **bottleneck**,** decoder** and **reconstruction loss**.
* As the name suggests, first the encoder learns to encode the data into a *compressed representation*.
* The bottleneck is basically the smallest encoded representation of the data.
* The decoder network learns how to *reconstruct the data* from the encoded network.
* The reconstruction loss is a measure of how well the image is being reconstructed and how close the output is to the original input.

![](https://www.jeremyjordan.me/content/images/2018/03/Screen-Shot-2018-03-09-at-10.20.44-AM.png)

* To learn more about autoencoders you can visit [this](http://ufldl.stanford.edu/tutorial/unsupervised/Autoencoders/)
* I would suggest going through the above website before proceeding further to get a better understanding of the code. 

# Building an Autoencoder

Constructing a very simple encoder and decoder network.
* You can see that our bottleneck is the Dense layer with 64 units.
* In the decoder network, the last layer gives us an output of 784 units using sigmoid activation. Sigmoid activation outputs either 1 or 0.
* Basically this means that the network will decide either to keep the noisy pixel or not. The output of 784 units are the pixels for out denoised image

In [ ]:
input_image = Input(shape = (784, ) )

encoded = Dense(512, activation = 'relu')(input_image)
encoded = Dense(512, activation = 'relu')(encoded)
encoded = Dense(256, activation = 'relu')(encoded)
encoded = Dense(256, activation = 'relu')(encoded)
encoded = Dense(64,  activation = 'relu')(encoded)

decoded = Dense(512, activation = 'relu')(encoded)
decoded = Dense(784, activation = 'sigmoid')(decoded)

autoencoder = Model(input_image, decoded)
autoencoder.compile(loss= 'binary_crossentropy' , optimizer = 'adam')
autoencoder.summary()

# Training Autoencoder

In [ ]:
#Lambda callback willl print the val_loss after each epoch
from tensorflow.keras.callbacks import LambdaCallback

autoencoder.fit(x_train_noisy, x_train,
               epochs = 100, batch_size = 512,
               validation_split = 0.2, verbose = False,
               callbacks=[LambdaCallback(on_epoch_end=lambda e,l: print('{:.3f}'.format(l['val_loss']), end=' _ '))]
               )

print("Training has finished !")

# Denoised Images

Now we will use the autoencoder to produce denoised images from noisy ones present in x_val

In [ ]:
preds = autoencoder.predict(x_val_noisy)


In [ ]:
plot(x_val, None)

In [ ]:
plot(x_val_noisy, None)

In [ ]:
#plotting the denoised images
plot(preds, None)

In [ ]:
loss, acc = classifier.evaluate(preds, y_val)
print(f"Loss : {loss} \nAccuracy : {acc}")

Earlier the accuracy was around 0.2 on images with noise.
As you can see our classifier performs well on denoised images. 

# Combined Model

Seperate models for noise reduction and classification are not very practical, hence we are going to combine them into single unit using the Model class.

In [ ]:
noisy_image = Input(shape = (784, ))
x = autoencoder(noisy_image)
y = classifier(x)
 
#combined model
denoise_and_classify = Model(noisy_image, y)

Making predictions using the combined network

In [ ]:
final_preds = denoise_and_classify.predict(x_val_noisy)

Plotting predicted class along with the images

In [ ]:
plot(x_val_noisy, final_preds , True)

Plotting actual class with images

In [ ]:
plot(x_val_noisy, to_categorical(y_val), True)

Making predinctions on our test data. First we convert it into a numpy array

Please note that data in test does not contain noise, for submitting and showing that our model works on test data too, we are making predictions using the complete model we have built

In [ ]:
test = test.to_numpy()

plot(test, None)

In [ ]:
test_preds = denoise_and_classify.predict(test)

results = np.argmax(test_preds, axis = 1)

results = pd.Series(results, name = 'Label')

In [ ]:
#The test images don't have noise but just for fun lets see what the autoencoder does to them
denoise_test = autoencoder.predict(test)
plot(denoise_test, None)


You can observe that autoencoder has somehow tried to *enhance* the images.

In [ ]:
#Plotting test predictions
plot(test,test_preds, True)

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("cnn_mnist_datagen.csv",index=False)